# How to work with AppEEARS CSV outputs in the cloud.

imported on: **2024-11-06**

<p>This notebook was originally developed by LP DAAC to show how to work with AppEEARS CSV outputs directly in the cloud.</p>

> The original source for this document is [https://github.com/nasa/AppEEARS-Data-Resources](https://github.com/nasa/AppEEARS-Data-Resources)

# Working with AρρEEARS Point Sample CSV outputs in AWS

This tutorial demonstrates how to access AρρEEARS point sample Comma-Separated Values (CSV) outputs direcly from the cloud. NASA's Application for Extracting and Exploring Analysis Ready Samples ([AρρEEARS](https://appeears.earthdatacloud.nasa.gov/)) has been migrated to NASA's Earthdata Cloud space located in **AWS us-west 2**. This enables users working in cloud instance deployed in **AWS us-west 2** to access outputs direcly in the cloud using S3 link returned by AρρEEARS. In this tutorial, we will walk through the process of submitting a point sample and accessing a CSV outputs from AρρEEARS.  

## Requirements  

- Earthdata Login Authentication is required to access the AρρEEARS API and AρρEEARS outputs. If you do not have an account, create an account [here](https://urs.earthdata.nasa.gov/users/new).  

## Learning Objectives  

- Learn how to access AρρEEARS point sample CSV outputs in AWS.  

## Tutorial Outline 

1. Setting up  
2. Submit a point request  
3. Extract the S3 links to data in S3  
4. Create a `boto3` Refreshable Session  
5. Direct S3 access of CSV output  
6. Quality Filtering  
7. Explore the LST time series   
    

## 1. Setting up  

Import required packages.

In [1]:
import requests
import numpy
import datetime
import os
import requests
from netrc import netrc
from pathlib import Path
from datetime import datetime, timezone
from botocore.client import Config
import pandas
import hvplot.pandas
import time
import warnings
import sys
sys.path.append('../modules/')
import aws_session
warnings.filterwarnings('ignore')

To successfully run this tutorial, it is required to create a **.netrc** file in your home directory. The function `_validate_netrc` defined in `aws_session` module checks if a properly formatted netrc file exists in your home directory. If the netrc file does not exist, it will prompt you for your Earthdata Login username and password and will create a netrc file. 

In [2]:
# validate if netrc file is present else create one via the user / password prompt for the urs.earthdata.nasa.gov
aws_session._validate_netrc()

INFO : netrc file is setup for urs.earthdata.nasa.gov already ...


## 2. Submit a point request in AρρEEARS 
In this step, we are going to submit a point request. You can also submit the same request to the [AρρEEARS Graphic User Interface (GUI)](https://appeears.earthdatacloud.nasa.gov/task/point) by uploading the JSON file provided in the repository (AppEEARS-Data-Resources/Data/point-request.json). If you have completed the request, save your `task_id` to a variable, skip this step, and move to the next step of tutorial.  

Assign the AρρEEARS API endpoint to a variable. 

In [3]:
appeears_API_endpoint = 'https://appeears.earthdatacloud.nasa.gov/api/'

A **Bearer Token** is needed to submit requests to the AρρEEARS API. To generated a token, a `POST` request containing Earthdata Login credentials stored in **.netrc** file is passed to the [`login`](https://appeears.earthdatacloud.nasa.gov/api/#authentication) service. 

In [4]:
urs = 'urs.earthdata.nasa.gov'

In [5]:
token_response = requests.post('{}login'.format(appeears_API_endpoint), auth = (netrc().authenticators(urs)[0],netrc().authenticators(urs)[2])).json() # Insert API URL, call login service, provide credentials & return json 
token_response

{'token_type': 'Bearer',
 'token': '3B4bfEhJGpIEEKwM8G8S7exFXRScIFLN8EmQ4xBF2NMtoWTppdJb8WV2ifMQqqNxvbeQsSN-N_TZTIwWybwYUA',
 'expiration': '2024-04-12T13:26:56Z'}

We'll save the **token** to an **Authorization** object we can pass in the header of any request made to the AppEEARS API.

In [6]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

The payload for an AppEEARS API request is a JSON object. Here, we compile the request JSON object with the required parameters. The coordinates (i.e., lat/lon pair) for where the LP DAAC is located are used as our point location. The data layers `LST_Night_1km` and `LST_Day_1km` from `MOD11A1` version 6.1 are selected below.

In [7]:
task_name = "EROS"
task_type = 'point'             # Type of task, area or point
startDate = '01-01-2015'        # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '03-01-2023'          # End of the date range for which to extract data: MM-DD-YYYY
lat =  [43.736038]
lon = [-96.62139]
prodLayer = [
    {"layer": "LST_Day_1km", "product": "MOD11A1.061"}, 
    {"layer": "LST_Night_1km", "product": "MOD11A1.061"}
]

In [8]:
task = {
    'task_type': task_type,
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'coordinates': [
             {
                 'latitude': lat[0],
                 'longitude': lon[0],
                 'id': "EROS" 
             }]        
    }
}

Now that we have our request object, or payload, create, we will now submit to AρρEEARS using the `post` function from `requests` library.

In [9]:
task_response = requests.post('{}task'.format(appeears_API_endpoint), json=task, headers=head).json()   # Post json to the API task service, return response as json
task_response

{'task_id': 'd32a4be3-ac6e-4247-afa0-a28bf57f46bd', 'status': 'pending'}

The `task_id` will be needed to get status information about the request and to later find the AρρEEARS outputs for the request. We will save the `task_id` to a variable and wait until our request is processed and complete. 

In [10]:
task_id = task_response['task_id']
task_id

'd32a4be3-ac6e-4247-afa0-a28bf57f46bd'

In [11]:
# Ping API until request is complete, then continue to Section 3
while requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'] != 'done':
    print(requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'])
    time.sleep(60)
print(requests.get('{}task/{}'.format(appeears_API_endpoint, task_id), headers=head).json()['status'])

queued
processing
processing
processing
processing
processing
done


## 3. Extract the Direct S3 links

Once our outputs are ready, we can get the information about our output files, also refered to as a bundle. If you submitted your request using AρρEEARS GUI, assign your sample's `task_id` to the variable `task_id` below. 

In [12]:
#task_id = '<ADD REQUEST ID HERE>'

`requests.get` is used to get information about our bundle. The bundle information includes `s3_url` in addition to the other information such as `file_name`, `file_id`, and `file_type`.  

Each output file can be downloaded using the `file_id` (see section 4 in [AppEEARS_API_Point.ipynb](https://github.com/nasa/AppEEARS-Data-Resources/blob/main/Python/tutorials/AppEEARS_API_Point.ipynb). Since AρρEEARS outputs are stored in an S3 bucket, outputs can also be accessed using `S3_url` if you are working from an cloud instance in **AWS us-west-2**. 

In [13]:
bundle = requests.get('{}bundle/{}'.format(appeears_API_endpoint,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
bundle

{'files': [{'sha256': '2d217eb9d967d849212bf6702a91e3fd615a7188cba72bf58e0b4d9849417865',
   'file_id': '71ee2606-8e24-4bc7-93a4-9e3431087ca5',
   'file_name': 'EROS-MOD11A1-061-results.csv',
   'file_size': 1168238,
   'file_type': 'csv',
   's3_url': 's3://appeears-output/008473a3-76f6-44e6-8c44-aae95f8d9df2/EROS-MOD11A1-061-results.csv'},
  {'sha256': '70dc8abc16c8368f6926f735f5f9057c27338a41f77165e648c600185acdff39',
   'file_id': '2c75ccff-430c-475a-9e37-50429bbe6cfc',
   'file_name': 'EROS-granule-list.txt',
   'file_size': 479520,
   'file_type': 'txt',
   's3_url': 's3://appeears-output/008473a3-76f6-44e6-8c44-aae95f8d9df2/EROS-granule-list.txt'},
  {'sha256': '7db425fb7a6f26c4a652226e587488a69e234b5d0ab98a6d79d24943abc77ea2',
   'file_id': '205e0462-cbde-4a52-b152-25b4a8971d57',
   'file_name': 'EROS-request.json',
   'file_size': 764,
   'file_type': 'json',
   's3_url': 's3://appeears-output/008473a3-76f6-44e6-8c44-aae95f8d9df2/EROS-request.json'},
  {'sha256': 'e5c1bad1bcbb

We can filter our bundle to only include the S3 URI for our CSV output. 

In [14]:
file = [f for f in bundle['files'] if f['file_type'] == 'csv']
csv_urls = file[0]['s3_url']
csv_urls

's3://appeears-output/008473a3-76f6-44e6-8c44-aae95f8d9df2/EROS-MOD11A1-061-results.csv'

## 4. Create a boto3 Refreshable Session

AρρEEARS outputs are freely accessible from a cloud instance in **AWS us-west-2**. In order to access our output files, a **Boto3 session** is needed. The Boto session will stores the required configurations that are passed between Python and AWS. Below, `get_boto3_refreshable_session` stored in `aws_session` will access your Earthdata Login credentidals store in .netrc file and generate temporary S3 credentials by making a call to AρρEEARS S3 credential endpoint and storing them in the boto3 session. This session will be auto-renewed as needed to prevent timeouts errors related to S3 credentials time limits.

In [15]:
region_name = 'us-west-2'
s3_creds_endpoint = f"{appeears_API_endpoint}/s3credentials"
# Boto3 Session required by the pandas package 
boto3_session = aws_session.get_boto3_refreshable_session(s3_creds_endpoint, region_name)


INFO : Getting new temporary credentials from AppEEARS API https://appeears.earthdatacloud.nasa.gov/api//s3credentials...


Next, an object for s3 client is created using `boto3_session`.


In [16]:
boto3_client = boto3_session.client('s3', config=Config(max_pool_connections=20))


## 5. CSV File In-Region Direct S3 Access 

To read the CSV file, an object is created for the bucket name along with the file name of csv file. AppEEARS stores the output csv file in a folder. `Bucket` and `key` can be extracted using `S3` URL as below.


In [17]:
csv_urls

's3://appeears-output/008473a3-76f6-44e6-8c44-aae95f8d9df2/EROS-MOD11A1-061-results.csv'

In [18]:
bucket_name = csv_urls.split('/')[2]
key_name = csv_urls.split(bucket_name)[1].strip("/")
obj = boto3_client.get_object(Bucket = bucket_name, Key = key_name) 

Next, `read_csv` from `pandas` is used to read the body of created object.  

In [19]:
output_csv = pandas.read_csv(obj['Body'])
output_csv

,ID,Latitude,Longitude,Date,MODIS_Tile,MOD11A1_061_Line_Y_1km,MOD11A1_061_Sample_X_1km,MOD11A1_061_LST_Day_1km,MOD11A1_061_LST_Night_1km,MOD11A1_061_QC_Day,...,MOD11A1_061_QC_Day_LST_Error_Flag_Description,MOD11A1_061_QC_Night_bitmask,MOD11A1_061_QC_Night_MODLAND,MOD11A1_061_QC_Night_MODLAND_Description,MOD11A1_061_QC_Night_Data_Quality_flag,MOD11A1_061_QC_Night_Data_Quality_flag_Description,MOD11A1_061_QC_Night_Emis_Error_flag,MOD11A1_061_QC_Night_Emis_Error_flag_Description,MOD11A1_061_QC_Night_LST_Error_Flag,MOD11A1_061_QC_Night_LST_Error_Flag_Description
0,EROS,43.736038,-96.62139,2015-01-01,h11v04,751.0,22.0,0.000000,0.000000,2.0,...,Average LST error <= 1K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
1,EROS,43.736038,-96.62139,2015-01-02,h11v04,751.0,22.0,268.940002,0.000000,65.0,...,Average LST error <= 2K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
2,EROS,43.736038,-96.62139,2015-01-03,h11v04,751.0,22.0,0.000000,0.000000,2.0,...,Average LST error <= 1K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
3,EROS,43.736038,-96.62139,2015-01-04,h11v04,751.0,22.0,256.259979,0.000000,65.0,...,Average LST error <= 2K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
4,EROS,43.736038,-96.62139,2015-01-05,h11v04,751.0,22.0,0.000000,0.000000,2.0,...,Average LST error <= 1K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2955,EROS,43.736038,-96.62139,2023-02-25,h11v04,751.0,22.0,264.540008,0.000000,0.0,...,Average LST error <= 1K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
2956,EROS,43.736038,-96.62139,2023-02-26,h11v04,751.0,22.0,270.299988,257.759979,0.0,...,Average LST error <= 1K,0b00000000,0b00,"LST produced, good quality, not necessary to e...",0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
2957,EROS,43.736038,-96.62139,2023-02-27,h11v04,751.0,22.0,0.000000,0.000000,2.0,...,Average LST error <= 1K,0b00000010,0b10,LST not produced due to cloud effects,0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K
2958,EROS,43.736038,-96.62139,2023-02-28,h11v04,751.0,22.0,0.000000,267.239990,2.0,...,Average LST error <= 1K,0b00000000,0b00,"LST produced, good quality, not necessary to e...",0b00,Good data quality,0b00,Average emissivity error <= 0.01,0b00,Average LST error <= 1K


## 6. Quality Filtering

To get the good quality data, the quality unique values are assigned to a variable. Good quality data have a decoded quality flag as `LST produced, good quality, not necessary to examine more detailed QA`. The LST values equal to zero, the layer fill_value, are also filtered. 

In [20]:
quality_val = numpy.array(output_csv['MOD11A1_061_QC_Night_MODLAND_Description'])
quality_unique = numpy.unique(quality_val)
quality_unique

array(['LST not produced due to cloud effects',
       'LST produced, good quality, not necessary to examine more detailed QA',
       'LST produced, other quality, recommend examination of more detailed QA'],
      dtype=object)

In [21]:
good_qual_flag = 'LST produced, good quality, not necessary to examine more detailed QA'
goodQual = output_csv[output_csv['MOD11A1_061_QC_Night_MODLAND_Description'] == good_qual_flag]
goodQual = goodQual[goodQual['MOD11A1_061_LST_Day_1km'] != 0]

## 6. Explore the LST Time Series 

The `hvplot` library can be used to create interactive visualization for `Pandas` `DataFrames`. First we will convert our dates in the **Date** column to `datetime` objects, then we will finish by plotting our good quality data values.

In [22]:
goodQual['dtDate'] = pandas.to_datetime(goodQual['Date'])

In [23]:
goodQual.hvplot.line(x='dtDate', 
                     y=['MOD11A1_061_LST_Day_1km', 'MOD11A1_061_LST_Night_1km'],
                     title='MODIS Day & Night Land Surface Temperature (LST)',
                     ylabel='LST (°C)', 
                     xlabel='Date',
                     xticks=12,
                     legend='bottom_right',
                     width=1200, 
                     height=600)

:NdOverlay   [Variable]
   :Curve   [dtDate]   (value)

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 04-19-2024  

¹Work performed under USGS contract 140G0121D0001 for NASA contract 80GSFC24TA009.   